In [ ]:
import transformers


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.1 MB/s eta 0:00:00


In [ ]:
import torch

In [ ]:
# Load a pre-trained transformer model and tokenizer
model = transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def sentiment(text):
  #tokenize input text
  input_ids = tokenizer.encode(text,return_tensors='pt')

  #generate output from the model 
  output= model(input_ids)

  #extract the hidden state of the last layer of the moidel
  last_hidden_state = output[0]

  #use a linear layer to map the hidden state to a single value
  linear_layer = torch.nn.Linear(last_hidden_state.shape[-1], 1)
  sentiment = torch.sigmoid(linear_layer(last_hidden_state.squeeze())).max()
  print(sentiment)
  if sentiment> 0.5:
    print('positive sentiment')
  elif sentiment <0.4:
    print('Nehative sentiment')
  else:
    print('Neutral Sentiment')



In [ ]:
sentiment("i hate very much")

tensor(0.6582, grad_fn=<MaxBackward1>)
positive sentiment


In [ ]:
sentiment("-")

tensor(0.4955, grad_fn=<MaxBackward1>)
Neutral Sentiment


In [ ]:
sentiment("shutuo")

tensor(0.5366, grad_fn=<MaxBackward1>)
positive sentiment


In [ ]:
sentiment("i hate you")

tensor(0.5936, grad_fn=<MaxBackward1>)
positive sentiment


In [ ]:
sentiment("i hate you very very very very very much")

tensor(0.6096, grad_fn=<MaxBackward1>)
positive sentiment


In [ ]:
sentiment("shutuo")

tensor(0.5268, grad_fn=<MaxBackward1>)
positive sentiment


---
---


In [ ]:
import numpy as np
from scipy.stats import mode
def emotions(text):
  #define the emotions to classify
  emotions=['happy','sad','angry','neutral']
  #tikenize input text
  input_ids = tokenizer.encode(text, return_tensors='pt')
  #generate output from the model 
  output= model(input_ids)

  #extract the hidden state of the last layer of the moidel
  last_hidden_state = output[0]
  #use a linear layer to map the hidden state to a prob.
  linear_layer = torch.nn.Linear(last_hidden_state.shape[-1],len(emotions)) 
  probabilities = torch.softmax(linear_layer(last_hidden_state.squeeze()).view(-1, len (emotions)),dim=1)

  #get the index of the higest probabilities
  predicted_index, count= mode(np.array([a.argmax().item() for a in probabilities]))
  print (predicted_index)
  predicted_emotion= emotions[predicted_index[0]]

  print(f'Predicted emotion:{predicted_emotion}')



In [ ]:
emotions("helloooooooo")

[0]
Predicted emotion:happy


In [ ]:
emotions("i am happy")

[0]
Predicted emotion:happy


In [ ]:
emotions("i am sad")

[2]
Predicted emotion:angry


In [ ]:
emotions("shut up")

[3]
Predicted emotion:neutral


---
---


In [ ]:
def summarize(text):
  summarization_pipeline= transformers.pipeline('summarization')#,model=model, tokenizer= tokenizer)
  
  #SUMMARIZE A TEXT
  summary = summarization_pipeline(text, min_length=10, max_length=50)[0].get("summary_text")
  print(f'Summary: {summary}')


In [ ]:
summarize("Lionel Messi is a professional soccer player who is widely considered one of the greatest players of all time. He has spent his entire career with FC Barcelona, leading the team to numerous domestic and international titles. Messi's skill on the pitch is unparalleled, and he is known for his lightning quick speed, exceptional ball control, and impressive goal-scoring abilities. He has won the Ballon d'Or, an award given to the best soccer player in the world, a record six times. Off the field, Messi is known for his charitable work and his quiet, unassuming demeanor. He is widely respected and admired by fans and fellow players alike for his talent and sportsmanship.")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Summary:  Lionel Messi is widely considered one of the greatest soccer players of all time . He has won the Ballon d'Or, an award given to the best soccer player in the world, a record six times . Off the field, Messi


---
---


In [ ]:
from sentence_transformers import SentenceTransformer , util
import spacy
from spacy.lang.en import STOP_WORDS
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re
import os
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!pip install sentence_transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f98c192a63a8b71c8c19213c7d2190b5f4c3c1cf8bb7174faf0cbf870f8f4232
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
def Similarity_Check(doc1, doc2):
  model = SentenceTransformer('roberta-base-nli-mean-tokens')

  query_embedding = model.encode(doc1)
  passage_embedding= model.encode(doc2)

  score= "{:.4f}".format(util.cos_sim(query_embedding, passage_embedding).tolist()[0][0])
  score_norm=(float(score)+1)/(1+1)
  print(f"Similarity Score with {score} :","{:.4f}".format(score_norm))
  return score_norm



In [ ]:
Similarity_Check("messi","ronaldo")

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/329 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Similarity Score with 0.6883 : 0.8441


0.84415

In [ ]:
Similarity_Check("Nepal", "Nepal")

Similarity Score with 1.0000 : 1.0000


1.0

In [ ]:
Similarity_Check("helloooooooooooo","helloooo")

Similarity Score with 0.9886 : 0.9943


0.9943